# Graphs for Competitive Analysis on New York City's Private For Hire Vehicle Companies 

Data Sources:

https://www.kaggle.com/datasets/shuhengmo/uber-nyc-forhire-vehicles-trip-data-2021

https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc

# Installing packages

In [1]:
!pip install my_package
!pip install pyarrow

import pandas as pd
import datetime as dt
import random

# Stratified Random Sampling
From an initial check of January 2021's trip data, the entries of High Volume For Hire Vehicle, which are for ride companies, like Uber, Lyft, and Via, reached 11.9 million rows. Therefore, a decision was made to do a strata random sampling on the data to study only 10% of the data. A strata technique was chosen so that the ride companies still have representation in the sampled data.

In [2]:
df1 = pd.read_parquet('fhvhv_tripdata_2021-01.parquet', engine='pyarrow')

In [3]:
df1.shape

(11908468, 24)

In [4]:
# Read parquet file
df = pd.read_parquet('fhvhv_tripdata_2021-01.parquet', engine='pyarrow')

# Get the unique values in the stratum column
strata_groups = df['hvfhs_license_num'].unique()
print(strata_groups)

['HV0003' 'HV0005' 'HV0004']


In [5]:
# Dictionary to store sample rows
sampled_rows = {}

# For loop to sample each group
for ride_company in strata_groups:
    stratum_rows = df.loc[df['hvfhs_license_num'] == ride_company]
    # Gets number of entries per ride company
    num_stratum_rows = stratum_rows.shape[0]
    # Gets number of samples we want, integer format
    num_sampled_rows = int(num_stratum_rows * 0.1)
    # Creates a list of indeces randomly selected
    sample_indices = random.sample(range(num_stratum_rows), num_sampled_rows)
    # Compiles randomly selected entries into a dataframe
    sampled_stratum_rows = stratum_rows.iloc[sample_indices]
    # Stores the random samples into the dictionary, using ride company as keys.
    sampled_rows[ride_company] = sampled_stratum_rows

# Stacking the random samples on top of each other to create the df
df = pd.concat(sampled_rows.values())

In [6]:
#Checking the df
df.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
10902816,HV0003,B02875,B02875,2021-01-29 17:53:00,2021-01-29 18:17:01,2021-01-29 18:17:01,2021-01-29 18:27:41,69,167,1.55,...,0.85,0.00,NaN,0.0,13.86,N,N,,N,Y
2138757,HV0003,B02878,B02878,2021-01-07 08:56:32,2021-01-07 08:58:20,2021-01-07 09:00:20,2021-01-07 09:17:24,47,126,2.25,...,1.47,0.00,NaN,0.0,12.56,N,N,,N,N
10133463,HV0003,B02764,B02764,2021-01-28 00:31:33,2021-01-28 00:33:00,2021-01-28 00:34:42,2021-01-28 00:39:47,219,10,1.52,...,0.56,0.00,NaN,0.0,5.39,N,N,,N,N
4668475,HV0003,B02835,B02835,2021-01-13 23:19:06,2021-01-13 23:23:54,2021-01-13 23:23:54,2021-01-13 23:33:55,76,63,1.87,...,0.70,0.00,NaN,0.0,7.09,N,N,,N,N
11196330,HV0003,B02682,B02682,2021-01-30 12:20:22,2021-01-30 12:24:16,2021-01-30 12:24:28,2021-01-30 12:33:16,79,249,1.74,...,0.91,2.75,NaN,0.0,6.34,N,N,,N,N


In [7]:
# Checking shape of df
df.shape

(1190845, 24)

# Data Cleaning

In [8]:
#Get a list of column names
df.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [9]:
#See data types per column
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1190845 entries, 10902816 to 580474
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   hvfhs_license_num     1190845 non-null  object        
 1   dispatching_base_num  1190845 non-null  object        
 2   originating_base_num  870211 non-null   object        
 3   request_datetime      1190845 non-null  datetime64[ns]
 4   on_scene_datetime     870829 non-null   datetime64[ns]
 5   pickup_datetime       1190845 non-null  datetime64[ns]
 6   dropoff_datetime      1190845 non-null  datetime64[ns]
 7   PULocationID          1190845 non-null  int64         
 8   DOLocationID          1190845 non-null  int64         
 9   trip_miles            1190845 non-null  float64       
 10  trip_time             1190845 non-null  int64         
 11  base_passenger_fare   1190845 non-null  float64       
 12  tolls                 1190845 non-nu

In [10]:
df.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [11]:
#Check missing values
df.isna().sum()

hvfhs_license_num             0
dispatching_base_num          0
originating_base_num     320634
request_datetime              0
on_scene_datetime        320016
pickup_datetime               0
dropoff_datetime              0
PULocationID                  0
DOLocationID                  0
trip_miles                    0
trip_time                     0
base_passenger_fare           0
tolls                         0
bcf                           0
sales_tax                     0
congestion_surcharge          0
airport_fee             1190238
tips                          0
driver_pay                    0
shared_request_flag           0
shared_match_flag             0
access_a_ride_flag            0
wav_request_flag              0
wav_match_flag                0
dtype: int64

In [12]:
#Dropping columns with missing values

#originating_base_num, on_scene_datetime, and airport_fee columns will be dropped since they have missing values.

df = df.drop(columns = ['originating_base_num','on_scene_datetime','airport_fee'])

In [13]:
#Check if dropped columns were dropped
df.isna().sum()

hvfhs_license_num       0
dispatching_base_num    0
request_datetime        0
pickup_datetime         0
dropoff_datetime        0
PULocationID            0
DOLocationID            0
trip_miles              0
trip_time               0
base_passenger_fare     0
tolls                   0
bcf                     0
sales_tax               0
congestion_surcharge    0
tips                    0
driver_pay              0
shared_request_flag     0
shared_match_flag       0
access_a_ride_flag      0
wav_request_flag        0
wav_match_flag          0
dtype: int64

In [14]:
# Check for duplicated rows
df[df.duplicated()]

# There are no duplicated rows.

,hvfhs_license_num,dispatching_base_num,request_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,...,bcf,sales_tax,congestion_surcharge,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
751647,HV0003,B02887,2021-01-03 00:33:05,2021-01-03 00:34:42,2021-01-03 00:48:30,71,39,3.2,828,13.24,...,0.4,1.18,0.0,0.0,10.46,N,N,,N,N


# Feature Engineering

In [15]:
# Drop unnecessary columns for the study.
df = df.drop(columns = ['dispatching_base_num','request_datetime','bcf','sales_tax','shared_request_flag','shared_match_flag','access_a_ride_flag','wav_request_flag','wav_match_flag','dropoff_datetime'])

In [16]:
# Checking which ride services are present in the dataset
df['hvfhs_license_num'].value_counts()

HV0003    870412
HV0005    309432
HV0004     11001
Name: hvfhs_license_num, dtype: int64

In [17]:
# Updating license numbers to ride company
# License numbers to their equivalent ride service providers are identified in the Kaggle PDF.

df['hvfhs_license_num'] = df['hvfhs_license_num'].replace(['HV0003', 'HV0005', 'HV0004'], ['Uber', 'Lyft', 'Via'])

In [18]:
# Creating a ride counter column for the dashboard
df['ride_count'] = 1

In [19]:
# Creating a date column for the transactions
df['Date']= df['pickup_datetime'].dt.date
df = df.drop(columns=['pickup_datetime'])

In [20]:
# Renaming columns
df = df.rename(columns={
    "hvfhs_license_num": "Ride Company",
    "trip_miles": "Trip Miles",
    "trip_time" : "Trip Time",
    "base_passenger_fare" : "Base Passenger Fare",
    "tolls" : "Tolls",
    "congestion_surcharge" : "Congestion Surcharge",
    "tips" : "Tips",
    "driver_pay" : "Driver Pay",
    "ride_count" : "Ride Count"
})

In [21]:
df.head()

,Ride Company,PULocationID,DOLocationID,Trip Miles,Trip Time,Base Passenger Fare,Tolls,Congestion Surcharge,Tips,Driver Pay,Ride Count,Date
10902816,Uber,69,167,1.55,640,9.63,0.0,0.00,0.0,13.86,1,2021-01-29
2138757,Uber,47,126,2.25,1024,16.56,0.0,0.00,0.0,12.56,1,2021-01-07
10133463,Uber,219,10,1.52,305,6.33,0.0,0.00,0.0,5.39,1,2021-01-28
4668475,Uber,76,63,1.87,601,7.84,0.0,0.00,0.0,7.09,1,2021-01-13
11196330,Uber,79,249,1.74,528,9.85,0.0,2.75,0.0,6.34,1,2021-01-30


In [22]:
df.to_csv('cleaned_data.csv')